In [1]:

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    pipeline
)
from typing import List
from datasets import load_dataset
from openai import AzureOpenAI
from sklearn.metrics import accuracy_score
from transformers import pipeline
import os
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sentence_transformers import SentenceTransformer
import json
import torch
from torch import cuda


/Users/aappopulkkinen/repos/llm-advanced-prompting-public/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "numind/NuExtract-tiny-v1.5"
device="mps"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [14]:
def get_prompt(text):
    template = """{
    "Model": {
        "Name": "",
        "Number of parameters": "",
        "Number of max token": "",
        "Architecture": []
    },
    "Usage": {
        "Use case": [],
        "Licence": ""
    }
}"""
    prompt = f"""<|input|>\n### Template:\n{template}\n### Text:\n{text}\n\n<|output|>"""
    return prompt

In [4]:
text = """We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and
automated benchmarks. Our models are released under the Apache 2.0 license.
Code: <https://github.com/mistralai/mistral-src>
Webpage: <https://mistral.ai/news/announcing-mistral-7b/>"""

In [16]:
get_prompt(text)

'<|input|>\n### Template:\n{\n    "Model": {\n        "Name": "",\n        "Number of parameters": "",\n        "Number of max token": "",\n        "Architecture": []\n    },\n    "Usage": {\n        "Use case": [],\n        "Licence": ""\n    }\n}\n### Text:\nWe introduce Mistral 7B, a 7–billion-parameter language model engineered for\nsuperior performance and efficiency. Mistral 7B outperforms the best open 13B\nmodel (Llama 2) across all evaluated benchmarks, and the best released 34B\nmodel (Llama 1) in reasoning, mathematics, and code generation. Our model\nleverages grouped-query attention (GQA) for faster inference, coupled with sliding\nwindow attention (SWA) to effectively handle sequences of arbitrary length with a\nreduced inference cost. We also provide a model fine-tuned to follow instructions,\nMistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and\nautomated benchmarks. Our models are released under the Apache 2.0 license.\nCode: <https://github

In [17]:
prompt_encoding = tokenizer(get_prompt(text), return_tensors="pt", truncation=True, padding=True, max_length=10000).to(model.device)
pred_ids = model.generate(**prompt_encoding, max_new_tokens=4000)
decoded_output = tokenizer.decode(pred_ids[0], skip_special_tokens=True)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [22]:
json.loads(decoded_output.split("<|output|>")[1])

{'Model': {'Name': 'Mistral 7B',
  'Number of parameters': '7–billion',
  'Number of max token': '',
  'Architecture': ['grouped-query attention (GQA)',
   'sliding window attention (SWA)']},
 'Usage': {'Use case': ['superior performance and efficiency',
   'reasoning',
   'mathematics',
   'code generation'],
  'Licence': 'Apache 2.0'}}